In [98]:
import os
import json
import numpy as np
import copy
import random
import trimesh
from utils.utils import *
from utils.trans  import InstanceObject, place_object_on, basic_transformation
import shutil
import torch
# surface_top_list = ['Mug','Hat','Bowl||','Lamp','Vase','Laptop','Keyboard','Bag','Scissors','Display','Knife','Bottle','Earphone']

# obejct_with_top_surface_list = ['StorageFurniture','Refrigerator', 'Table','Bed', 'Dishwasher']

# obejct_without_top_surface_list =['TrashCan','Chair','Door']



surface_top_list = ['Mug','Lamp','Vase','Laptop','Keyboard','Bag','Display','Bottle']
# No Knife,Hat,'Display','Scissors','Earphone','Bowl' for now 

obejct_with_top_surface_list = ['StorageFurniture','Refrigerator', 'Table','Bed', 'Dishwasher']

obejct_without_top_surface_list =['TrashCan','Chair','Door']


In [99]:
def basic_transformation_v2(mesh,obbs, scale_factor):
    rotation_matrix_x = create_rotation_matrix('x', 90)
    mesh.apply_transform(rotation_matrix_x)
    obbs = apply_transform_to_obbs(obbs, rotation_matrix_x)

    translation_to_ground = create_translation_matrix([0, 0, -mesh.bounds[0][2]])
    mesh.apply_transform(translation_to_ground)
    obbs = apply_transform_to_obbs(obbs, translation_to_ground)

    scale_matrix = create_scale_matrix(scale_factor)
    mesh.apply_transform(scale_matrix)
    obbs = apply_transform_to_obbs(obbs, scale_matrix)
    
    return mesh,obbs

def apply_transform_to_obbs(obbs, transform_matrix):
    transformed_obbs = {}
    for key, obb in obbs.items():
        obb_homogeneous = np.hstack((obb, np.ones((2, 1))))
        transformed_obb = np.dot(transform_matrix, obb_homogeneous.T).T[:, :3]
        transformed_obbs[key] = transformed_obb
    return transformed_obbs


In [100]:
from utils.trans import *
import numpy as np
import copy
import random

# Define the function to place objects on surfaces
def place_object_on_v2(source_object, target_object, large_object_surface_obb):
    surface_bounds = large_object_surface_obb
    surface_bounds_min_x, surface_bounds_min_y = surface_bounds[0][0], surface_bounds[0][1]
    surface_bounds_max_x, surface_bounds_max_y = surface_bounds[1][0], surface_bounds[1][1]
    surface_height = surface_bounds[1][2]
    
    source_object_ = copy.deepcopy(source_object)
    
    retry_count = 0
    while True:
        retry_count += 1
        if retry_count > 5:
            raise Exception("Too many retries")
        
        random_x = random.uniform(surface_bounds_min_x, surface_bounds_max_x)
        random_y = random.uniform(surface_bounds_min_y, surface_bounds_max_y)
        place_at = [random_x, random_y, surface_height + 0.0001]

        angles = [180, 150, 120, 90, 60, 30, 0, -30, -60, -90, -120, -150, -180]
        random_rotation = random.choice(angles)

        source_object_.position = place_at
        source_object_.rotation = random_rotation
        source_object_.apply_transformation()

        if target_object.overlap_detection(source_object_.obbs):
            source_object_ = copy.deepcopy(source_object)
            continue

        overlap = False
        for obj in target_object.objects_on_surface.values():
            if obj.overlap_detection(source_object_.obbs):
                overlap = True
                source_object_ = copy.deepcopy(source_object)
                break

        if not overlap:
            source_object.position = place_at
            source_object.rotation = random_rotation
            source_object.apply_transformation()
            target_object.objects_on_surface[source_object.category] = source_object
            
            break


In [101]:
from utils.trans import *
import numpy as np
import copy

class InstanceObject_v2:
    def __init__(self, obj_id, category, instance_mesh, scale_factor, obbs, orientation=np.array([0, 1, 0])):
        self.obj_id = obj_id
        self.category = category
        self.mesh, self.obbs = basic_transformation_v2(instance_mesh,obbs, scale_factor )
        self.scale_factor = scale_factor
        self.orientation = orientation

        # Initialize position and transformation attributes
        self.position = np.zeros(3)
        self.rotation_matrix = np.eye(4)
        self.translation_matrix = np.eye(4)
        
        # Extract mesh details
        self.central_points = {}
        self.bounds = self.mesh.bounds
        self.height = self.bounds[1][2] - self.bounds[0][2]
        self.part_list = []  # Store all the parts information

        # Additional attributes
        self.rotation = 0
        self.transformation = None
        self.objects_on_surface = {}  # Objects on the surfaces

    def apply_transformation(self):
        final_translation_matrix = create_translation_matrix(self.position)
        center_translation_to_origin = create_translation_matrix(-self.mesh.centroid)
        center_translation_back = create_translation_matrix(self.mesh.centroid)
        final_transform = np.dot(
            center_translation_back,
            np.dot(create_rotation_matrix('z', self.rotation), center_translation_to_origin)
        )
        final_transform = np.dot(final_translation_matrix, final_transform)
        self.mesh.apply_transform(final_transform)
        self.transformation = final_transform
        self.obbs = apply_transform_to_obbs(self.obbs, final_transform)
        return final_transform

    def get_surface(self):
        surface_nodename_map = {
            'Bed': 'bed:mattress_0',
            'Table': 'table:board_0'
        }
        surface_nodename = surface_nodename_map.get(self.category, None)
        if surface_nodename is None:
            raise ValueError(f"No surface node defined for category {self.category}")
        top_obb = self.obbs[surface_nodename]
        return top_obb


        #     def get_surface_mesh(self):

        # surface_nodename = surface_nodename_map[self.category]
        # self.obbs
        # return top_obb
        
        
    def overlap_detection(self, other_obbs):
        for key, self_obb in self.obbs.items():
            self_min, self_max = self_obb[0], self_obb[1]
            for other_key, other_obb in other_obbs.items():
                other_min, other_max = other_obb[0], other_obb[1]
                intersects = not (
                    self_max[0] < other_min[0] or self_min[0] > other_max[0] or  # x-axis
                    self_max[1] < other_min[1] or self_min[1] > other_max[1] or  # y-axis
                    self_max[2] < other_min[2] or self_min[2] > other_max[2]    # z-axis
                )
                if intersects:
                    print('Overlap')
                    return True
        return False

    def get_obbs(self):
        return self.obbs

    def get_central_points_from_obbs(self):
        for key, value in self.obbs.items():
            central_point = (value[0] + value[1]) / 2
            self.central_points[key] = central_point
        return self.central_points


In [110]:
cat_factors = {
    'Mug': (0.2, 0.3),        # Mugs typically vary a bit in size but are generally small.
    'Hat': (0.2, 0.3),        # Hats have some variation but are generally smaller.
    'Bowl': (0.2, 0.3),       # Bowls can range from small to medium sizes.
    'Lamp': (0.6, 0.8),       # Lamps can range from small desk lamps to larger ones.
    'Vase': (0.4, 0.5),       # Vases can vary significantly in size but are usually not as large as laptops.
    'Laptop': (0.6, 0.8),     # Laptops have a narrow size range and are relatively larger.
    'Keyboard': (0.4, 0.7),   # Keyboards also have a narrow size range similar to laptops.
    'Bag': (0.5, 1.0),        # Bags can vary significantly in size.
    'Scissors': (0.2, 0.5),   # Scissors typically have a limited size range and are generally small.
    'Display': (0.7, 1.0),    # Displays usually have a narrow size range and are relatively larger.
    'Knife': (0.2, 0.5),      # Knives can vary from small to medium sizes.
    'Bottle': (0.2, 0.5),     # Bottles can vary significantly in size.
    'Earphone': (0.2, 0.5)    # Earphones are generally very small.
}

In [120]:
config = load_json('scene_layout.json')
all_obj_dict = load_json('obj_correspondence_tiny.json')
large_id_list = [obj['obj_id'] for obj in config['large_objects']]
large_objects_contain = {}
tiny_obj_id_list = []
obj_num = 1
for obj_id in [obj['obj_id'] for obj in config['large_objects']]:
    tiny_objects = []
    for i in range(obj_num):
        # still random
        tiny_obj_cat, tiny_obj_id= random_objects_generate(surface_top_list,all_obj_dict)
        tiny_obj_info = {
            "obj_id":tiny_obj_id,
            "category":tiny_obj_cat,
            "scale_factor":random.uniform(cat_factors[tiny_obj_cat][0],cat_factors[tiny_obj_cat][1])
        }
        tiny_objects.append(tiny_obj_info)
        tiny_obj_id_list.append(tiny_obj_id)
    large_objects_contain[obj_id] = tiny_objects
id_list = large_id_list+tiny_obj_id_list

# 读取所有的点云

In [121]:
def load_seg(filename):
    """Read and return the contents of the specified file as a list of integers."""
    with open(filename, 'r') as file:
        lines = file.readlines()
        return [int(line.strip()) for line in lines]
mesh_dict = {}
point_cloud ={}
seg_labels = []
seg_dict = {}
cat_to_id = {}
obbs_dict = {}
# load obj and obj information
for obj_id in id_list:
        # print(obj_id)
        obj_base_dir = '/home/lidosan/Datasets/PartNet_complete/{}'.format(obj_id)
        obj_structure = load_json(os.path.join(obj_base_dir, 'label_to_number.json'))
        obj_mesh = trimesh.load(os.path.join(obj_base_dir,f"{obj_id}.obj"))
        obj_npy = np.load(os.path.join(obj_base_dir,'points.npy'))
        obj_cat = load_json(os.path.join(obj_base_dir, 'meta.json'))['model_cat']
        seg_label = load_seg(os.path.join(obj_base_dir,'seg_label.txt'))
        obbs = torch.load(os.path.join(obj_base_dir, 'obbs.pth'))
        seg_labels = seg_labels + seg_label 
        mesh_dict[obj_id] = obj_mesh
        seg_dict[obj_id] = seg_label
        cat_to_id[obj_id] = obj_cat
        obbs_dict[obj_id] = obbs
        # 可以等所有的标签确定好以后再生成GT，直接映射就好了
        # return a total，mask

# Initialize instances
instances = {}
cat_list =[]
import uuid

# Without clock

# TODO add obbs to parts and over lap detection

    
for large_object in config['large_objects']:
    lid = large_object['obj_id']
    mesh = copy.deepcopy(mesh_dict[lid])
    # save the uv_mesh here 
    label_count = 0
    instance_id = str(uuid.uuid4())[:3]
    instance = InstanceObject_v2(lid, cat_to_id[lid], mesh, large_object['scale_factor'],obbs_dict[lid])
    instance.position = large_object['position']
    instance.rotation = large_object['rotation']
    instance.apply_transformation()
    instances[f'{instance.category}_{instance_id}_{lid}'] = instance
    try:
        large_object_surface_obb = instance.get_surface()
    except:
        continue
    
    for tiny_object in large_objects_contain[lid]:
        # print('Tiny',tiny_object['obj_id'])
        tid = tiny_object['obj_id']
        mesh = copy.deepcopy(mesh_dict[tiny_object['obj_id']])
        instance_id = str(uuid.uuid4())[:3]
        tiny_instance = InstanceObject_v2(tid, cat_to_id[tid], mesh, tiny_object['scale_factor'],obbs_dict[tid])
        try:
            place_object_on_v2(tiny_instance, instance, large_object_surface_obb)
        except:
            continue
        instances[f'{tiny_instance.category}_{instance_id}_{tid}'] = tiny_instance




Overlap


In [122]:
import trimesh.scene

objects_mseh_list = [instance.mesh for instance in instances.values()]
objects_mesh_uv_list = []
tiny_combined_scene = trimesh.Scene()
# Instance level scene
instance_level_scene = trimesh.scene.scene.append_scenes(objects_mseh_list)
instance_level_scene.show(viewer='gl')

SceneViewer(width=1800, height=1350)